# **Capstone project: Providing data-driven suggestions for HR**

## Description and deliverables

This capstone project is about deploying predictive model that can provide insights to the Human Resources (HR) department of a large consulting firm. This implementation only covers  deployment of a model, which derived from data analysis process that undergo certain steps as describe in CRISP-DM framework.




In [ ]:
# Splitting data in training and testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y, test_size=0.25,random_state=42)

In [ ]:
# Using logistic regression
from sklearn.linear_model import LogisticRegression
log_mod= LogisticRegression(random_state=42,max_iter=500)

In [ ]:
# Fittng model
log_mod.fit(X_train,y_train)

#Predicting value
y_pred=log_mod.predict(X_test)


In [ ]:
Pkl_Filename_log_mod = "model_log_mod.pkl"

with open(Pkl_Filename_log_mod, 'wb') as file:
    pickle.dump(log_mod, file)

In [ ]:
from pydantic import BaseModel

class Logmod(BaseModel):
    satisfaction_level: float
    last_evaluation: float
    number_project: int
    average_monthly_hours	: int
    time_spend_company: int
    work_accident	: int
    promotion_last_five_years: int
    salary: int
    department_IT: bool
    department_RandD: bool
    department_accounting: bool
    department_hr: bool
    department_management: bool
    department_marketing: bool
    department_product_mng: bool
    department_sales: bool
    department_support: bool
    department_technical: bool
    class Config:
        json_schema_extra = {
            "example": {
                "satisfaction_level": 0.38,
                "last_evaluation": 0.53,
                "number_project": 2,
                "average_monthly_hours": 157,
                "time_spend_company": 3,
                "work_accident": 0,
                "promotion_last_five_years": 1,
                "salary": 1
            }
        }



In [ ]:
#!pip install colabcode

In [ ]:
!pip install fastapi

In [ ]:
!pip install uvicorn

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
!ngrok authtoken <your token>
ngrok_tunnel = ngrok.connect(8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from fastapi import FastAPI
import json
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
app = FastAPI()
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post('/predict')
def get_employee_attrition(input_parameters : Logmod):

    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    satisfaction_level = input_dictionary['satisfaction_level']
    last_evaluation = input_dictionary['last_evaluation']
    number_project = input_dictionary['number_project']
    average_monthly_hours = input_dictionary['average_monthly_hours']
    time_spend_company = input_dictionary['time_spend_company']
    work_accident = input_dictionary['work_accident']
    promotion_last_five_years = input_dictionary['promotion_last_five_years']
    salary = input_dictionary['salary']
    department_IT = input_dictionary['department_IT']
    department_RandD = input_dictionary['department_RandD']
    department_accounting = input_dictionary['department_accounting']
    department_hr = input_dictionary['department_hr']
    department_management = input_dictionary['department_management']
    department_marketing = input_dictionary['department_marketing']
    department_product_mng = input_dictionary['department_product_mng']
    department_sales = input_dictionary['department_sales']
    department_support = input_dictionary['department_support']
    department_technical = input_dictionary['department_technical']


    input_list = [satisfaction_level, last_evaluation, number_project,
                  average_monthly_hours, time_spend_company, work_accident, promotion_last_five_years,salary,
                  department_IT,department_RandD,department_accounting,department_hr,department_management,
                  department_marketing,department_product_mng,department_sales,department_support,
                  department_technical]

    prediction = log_mod.predict([input_list])

    if (prediction[0] == 0):
        return 'The person might not left the company'
    else:
        return 'The person might left the company'

uvicorn.run(app, port=8000)